In [6]:
import psycopg2
import pandas as pd
import numpy as np

In [7]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="postgres"
)

In [8]:
cur = conn.cursor()

In [9]:
cur.execute("""

        select bank_id, transaction_type_chain, number_of_chain, swtc+swtv total, wight
        from(
            select bank_id, transaction_type_chain, number_of_chain, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, transaction_type_chain, number_of_chain, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, transaction_type_chain, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1 number_of_chain,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, transaction_type_chain, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, transaction_type_chain, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1 number_of_chain,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, transaction_type_chain, number_of_chain, transaction_cost, transaction_value) as b3
            group by bank_id, number_of_chain, transaction_type_chain) as b4
            where bank_id = 13335 and number_of_chain = (    select number_of_chain
    from(
        select bank_id, number_of_chain, swtc+swtv total, wight
        from(
            select bank_id, number_of_chain, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, number_of_chain, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1 number_of_chain,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1 number_of_chain,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, number_of_chain, transaction_cost, transaction_value) as b3
            group by bank_id, number_of_chain) as b4
            where bank_id = 13335
            order by total desc
            limit 1) as b5)
            order by total desc
            """)

In [10]:
result = cur.fetchall()
pd.DataFrame(result)

,0,1,2,3,4
0,13335,28392p;28691p;17s;28391m;27333m;27714m,6,18537082219,16703
1,13335,28392p;28691p;6s;28391m;27333m;1122s,6,8571626945,166205
2,13335,28392p;28691p;6s;1122s;28391m;27333m,6,2476618286,43371
3,13335,28392p;28691p;6s;28391m;27333m;847s,6,1154107386,13193
4,13335,28392p;6s;1122s;28391m;27333m;28412p,6,952222386,8903
...,...,...,...,...,...
78,13335,28392p;28691p;27525p;28454p;6s;27714m,6,750,12
79,13335,28392p;28691p;27525p;28497m;6s;27714m,6,740,6
80,13335,28392p;28691p;28481m;3501s;27714m;28412p,6,636,1
81,13335,28392p;28691p;6s;27714m;6s;1002s,6,628,1


In [11]:
data = pd.read_csv('tranasaction_dataset.csv', parse_dates=['time_internal'])
data = data.replace(np.nan,0)
data['length_chain'] = data['transaction_type_chain'].str.count(';')+1

In [12]:
tmp_data = data[data['count'] == 0]
dg1 = tmp_data.groupby(by=['bank_id', 'length_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [13]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'length_chain', 'transaction_cost', 'transaction_value']]

In [14]:
dg = pd.concat([dg1, dg2])

In [15]:
dg = dg.groupby(by=['bank_id', 'length_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [16]:
dg = dg.groupby(by=['bank_id', 'length_chain']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [17]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [18]:
your_bank_id = 13335
final = dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False).head(1)
best = final['length_chain'].values[0]

In [19]:
data = data[data['length_chain'] == best]

In [20]:
tmp_data = data[data['count'] == 0]
dg1 = tmp_data.groupby(by=['bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [21]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']]

In [22]:
dg = pd.concat([dg1, dg2])

In [23]:
dg = dg.groupby(by=['bank_id', 'transaction_type_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [24]:
dg = dg.groupby(by=['bank_id', 'transaction_type_chain']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [25]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [26]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

,bank_id,transaction_type_chain,count,total_value
2818,13335,28392p;28691p;17s;28391m;27333m;27714m,16703,18537082219
2859,13335,28392p;28691p;6s;28391m;27333m;1122s,166205,8571626945
2853,13335,28392p;28691p;6s;1122s;28391m;27333m,43371,2476618286
2865,13335,28392p;28691p;6s;28391m;27333m;847s,13193,1154107386
2874,13335,28392p;6s;1122s;28391m;27333m;28412p,8903,952222386
...,...,...,...,...
2819,13335,28392p;28691p;27525p;28454p;6s;27714m,12,750
2821,13335,28392p;28691p;27525p;28497m;6s;27714m,6,740
2830,13335,28392p;28691p;28481m;3501s;27714m;28412p,1,636
2857,13335,28392p;28691p;6s;27714m;6s;1002s,1,628
